# Sampling RDDs

#### [Baseado em "Introduction to Spark with Python, by Jose A. Dianes"](https://github.com/jadianes/spark-py-notebooks)

Até agora nós introduzimos a criação de RDD junto com algumas transformações básicas como `map` e` filter` e algumas ações como `count`,` take` e `collect`.

Este notebook mostrará como amostrar RDDs. Em relação às transformações, o `sample` será introduzido, uma vez que será útil em muitos cenários de aprendizagem estatística. Então, vamos comparar os resultados com a ação `takeSample`.

## Fazendo download do Dataset

Neste notebook, usaremos o conjunto de dados reduzido (10 por cento) fornecido para a KDD Cup 1999, contendo quase meio milhão de interações de rede. O arquivo é fornecido como um arquivo *Gzip* que será baixado localmente.

In [1]:
from urllib.request import urlretrieve

f = urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz", "kddcup.data.gz")

## Obtendo os dados e criando o RDD

Neste caso, usaremos o conjunto de dados completo fornecido para a KDD Cup 1999, contendo quase meio milhão de interações de rede. O arquivo é fornecido como um arquivo Gzip que será baixado localmente.

Agora podemos usar esse arquivo para criar nosso RDD.

In [2]:
data_file = "./kddcup.data.gz"
raw_data = sc.textFile(data_file)

## Sampling RDDs   

No Spark, há duas operações de amostragem, a transformação `sample` e a ação` takeSample`. Ao usar uma transformação, podemos dizer ao Spark para aplicar uma transformação sucessiva em uma amostra de um determinado RDD. Ao usar uma ação, recuperamos uma amostra e podemos armazená-la na memória local para ser usada por qualquer outra biblioteca padrão (por exemplo, Scikit-learn).

### A transformação `sample`

A transformação `sample` possui até três parâmetros. A primeira é se a amostragem é feita com substituição ou não. Em segundo lugar, o tamanho da amostra é uma fração. Por fim, podemos opcionalmente fornecer um *random seed*.

In [3]:
raw_data_sample = raw_data.sample(False, 0.1, 1234)
sample_size = raw_data_sample.count()
total_size = raw_data.count()
print("Sample size is {} of {}".format(sample_size, total_size))

Sample size is 489957 of 4898431


Mas o poder da amostragem como uma transformação vem de fazê-lo como parte de uma sequência de transformações adicionais. Isso será mais poderoso quando começarmos a fazer operações de agregações e pares de valores-chave e será especialmente útil ao usar a biblioteca de aprendizado de máquina do Spark MLlib.

Enquanto isso, imagine que queremos ter uma aproximação da proporção de interações `normal` em nosso conjunto de dados. Poderíamos fazer isso contando o número total de tags como fizemos nos cadernos anteriores. No entanto, queremos uma resposta mais rápida e não precisamos da resposta exata, mas apenas uma aproximação. Nós podemos fazer isso da seguinte maneira.

In [8]:
from time import time

# transformations to be applied
raw_data_sample_items = raw_data_sample.map(lambda x: x.split(","))
sample_normal_tags = raw_data_sample_items.filter(lambda x: "normal." in x)

# actions + time
t0 = time()
sample_normal_tags_count = sample_normal_tags.count()
tt = time() - t0

sample_normal_ratio = sample_normal_tags_count / float(sample_size)
print("The ratio of 'normal' interactions is {}".format(round(sample_normal_ratio,3)))
print("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions is 0.199
Count done in 26.617 seconds


Vamos comparar isso com o cálculo da proporção sem amostragem.

In [9]:
# transformations to be applied
raw_data_items = raw_data.map(lambda x: x.split(","))
normal_tags = raw_data_items.filter(lambda x: "normal." in x)

# actions + time
t0 = time()
normal_tags_count = normal_tags.count()
tt = time() - t0

normal_ratio = normal_tags_count / float(total_size)
print("The ratio of 'normal' interactions is {}".format(round(normal_ratio,3)))
print("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions is 0.199
Count done in 44.457 seconds


Nós podemos ver um ganho no tempo. Quanto mais transformações aplicarmos após a amostragem, maior será esse ganho. Isso porque, sem amostrar, todas as transformações são aplicadas ao conjunto completo de dados.

### A ação `takeSample`

Se o que precisamos é pegar uma amostra de dados brutos do nosso RDD na memória local para ser usado por outras bibliotecas não-Spark, o `takeSample` pode ser usado.

A sintaxe é muito semelhante, mas neste caso especificamos o número de itens em vez do tamanho da amostra como uma fração do tamanho completo dos dados.

In [10]:
t0 = time()
raw_data_sample = raw_data.takeSample(False, 400000, 1234)
normal_data_sample = [x.split(",") for x in raw_data_sample if "normal." in x]
tt = time() - t0

normal_sample_size = len(normal_data_sample)

normal_ratio = normal_sample_size / 400000.0
print("The ratio of 'normal' interactions is {}".format(normal_ratio))
print("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions is 0.1988025
Count done in 48.52 seconds


O processo foi muito parecido como antes. Obtivemos uma amostra de cerca de 10% dos dados e depois filtramos e dividimos.

No entanto, demorou mais tempo, mesmo com uma amostra ligeiramente menor. O motivo é que o Spark acabou de distribuir a execução do processo de amostragem. A filtragem e divisão dos resultados foram feitas localmente em um único nó.